In [361]:
import bs4
import time
from time import sleep
import pandas as pd
from io import StringIO
import pandas as pd
import re
import requests

In [386]:
def scrape_course(page):
    soup = bs4.BeautifulSoup(page.text)
    course_numbers = []
    descriptions = []
    terms_offered = []
    equivalent_courses = []
    prerequisites = []
    instructors = []
    
    course_blocks = soup.find_all("div", class_="courseblock main")
    subsequence_blocks = soup.find_all("div", class_="courseblock subsequence")
    
    for course in course_blocks+subsequence_blocks:
        if course.find("p", class_="courseblockdetail"):
            course_number = course.find("p", class_="courseblocktitle").strong.text.split('.')[0]
            if len(course_number) == 10:
                course_numbers.append(course_number[:4]+ " " + course_number[-5:])
            else:
                course_number.append(course_number[:4]+ ' ' + course_number[5:])
            description = course.find("p", class_="courseblockdesc")
            if description:
                course_description = description.text.strip()
                descriptions.append(course_description)
            else:
                descriptions.append(None)
            course_details = course.find("p", class_="courseblockdetail").text.strip()
            term_offered = re.findall(r"Terms Offered: (.+)", course_details)
            if term_offered:
                terms_offered.append(term_offered[0])
            else:
                terms_offered.append(None)
            instructor = re.findall(r"Instructor\(s\): ([^\s][^:]+?)(?:\s*(?:Terms Offered|Prerequisite|Equivalent Course)\(s\)|$)", course_details)
            if instructor:
                instructors.append(instructor[0])
            else:
                instructors.append(None)
            equivalent_course = re.findall(r"Equivalent Course\(s\): (.+?)(?:\n|$)", course_details)
            if equivalent_course:
                equivalent_courses.append(equivalent_course[0])
            else:
                equivalent_courses.append(None)
            prerequisite = re.findall(r"Prerequisite\(s\): (.+?)\n", course_details)
            if prerequisite:
                prerequisites.append(prerequisite[0])
            else:
                prerequisites.append(None)
    df = pd.DataFrame({
        'Course Number': course_numbers,
        'Description': descriptions,
        'Terms Offered': terms_offered,
        'Equivalent Courses': equivalent_courses,
        'Prerequisites': prerequisites,
        'Instructors': instructors
    })
    return df

def process_li(li):
    href = li.find('a')['href']
    full_url = "http://collegecatalog.uchicago.edu" + href
    print(full_url)
    page = requests.get(full_url)
    if page.status_code == 200:
        return scrape_course(page)
    else:
        print(f"Failed to retrieve page: {full_url}")
        return pd.DataFrame()
    
page = requests.get("http://collegecatalog.uchicago.edu/thecollege/programsofstudy/")
soup = BeautifulSoup(page.text)
li_elements = soup.find_all('li')
dfs = []
for li in li_elements[6:-28]:
    df = process_li(li)
    if not df.empty:
        dfs.append(df)
    sleep(3)
final_df = pd.concat(dfs, ignore_index=True)
final_df

http://collegecatalog.uchicago.edu/thecollege/anthropology/
http://collegecatalog.uchicago.edu/thecollege/architecturalstudies/
http://collegecatalog.uchicago.edu/thecollege/arthistory/
http://collegecatalog.uchicago.edu/thecollege/astronomyastrophysics/
http://collegecatalog.uchicago.edu/thecollege/biologicalchemistry/
http://collegecatalog.uchicago.edu/thecollege/biologicalsciences/
http://collegecatalog.uchicago.edu/thecollege/chemistry/
http://collegecatalog.uchicago.edu/thecollege/cinemamediastudies/
http://collegecatalog.uchicago.edu/thecollege/classicalstudies/
http://collegecatalog.uchicago.edu/thecollege/cognitivescience/
http://collegecatalog.uchicago.edu/thecollege/comparativehumandevelopment/
http://collegecatalog.uchicago.edu/thecollege/comparativeliterature/
http://collegecatalog.uchicago.edu/thecollege/caam/
http://collegecatalog.uchicago.edu/thecollege/computerscience/
http://collegecatalog.uchicago.edu/thecollege/creativewriting/
http://collegecatalog.uchicago.edu/thec

,Course Number,Description,Terms Offered,Equivalent Courses,Prerequisites,Instructors
0,ANTH 10100,Classically defined as the 'science of humanki...,None,None,None,Staff
1,ANTH 10107,The crisis of nature and technology is at the ...,None,"HIPS 20107, CEGU 20107",None,None
2,ANTH 12800,Whose land are we on? What does it mean to be ...,Autumn,"CRES 12800, CHST 12300, HIST 17800, RDIN 12300",None,None
3,ANTH 20006,This course approaches bodies as points of ins...,None,"GNSE 24006, RLST 26006",None,Mareike Winchell
4,ANTH 20537,"In America, the Black Panther Party and its le...",Spring,"RDIN 26674, RLST 26674, GLST 26674",None,None
...,...,...,...,...,...,...
4897,YDDH 22323,In this course we will make our way through is...,Autumn,"JWSC 27652, YDDH 32323",None,None
4898,YDDH 25524,"In this course, we will examine the Yiddish re...",Winter,"YDDH 35524, JWSC 27653",None,None
4899,YDDH 10100,The goal of this sequence is to develop profic...,Autumn,JWSC 20300,None,None
4900,YDDH 10200,"In this course, students will extend basic Yid...",Winter,"JWSC 20400, YDDH 37400",YDDH 10100,None


In [394]:
final_df["Department"]=final_df["Course Number"].str[:4]
final_df.groupby("Department")["Course Number"].count().idxmax()

'ENGL'

In [411]:
final_df['Terms Offered'] = final_df['Terms Offered'].fillna('None')
quarters = final_df['Terms Offered'].str.split(',').explode().str.strip()
quarter_counts = quarters.value_counts()
quarter_counts

Terms Offered
Autumn                                                     1346
Spring                                                     1244
Winter                                                     1243
None                                                        691
TBD                                                         101
                                                           ... 
Spring. Every other spring. Not offered in Spring 2021.       1
Spring. Course will be taught spring 2021                     1
Autumn. Not offered in 2021-2022.                             1
Winter. Not offered 21-22.                                    1
To be determined                                              1
Name: count, Length: 117, dtype: int64

In [412]:
final_df.to_csv('HW6_dataframe.csv', index=False)